In [ ]:
!pip install transformers librosa soundfile

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
# JavaScript code to create buttons and implement audio recording functionality
MAKE_RECORD_BUTTONS_JS = """
(async () => {
  let recorderDiv = document.getElementById('recorderDiv');

  if (recorderDiv) return;

  // ===================================
  // ========== Make buttons ===========

  recorderDiv = document.createElement('div');
  recorderDiv.id = 'recorderDiv';

  document.body.appendChild(recorderDiv);

  recorderDiv.innerHTML = `
    <button id='start'>Start Recording</button>
    <button id='stop' disabled>Stop Recording</button>
  `;

  const startButton = document.getElementById('start');
  const stopButton = document.getElementById('stop');

  // ===================================
  // ===================================

  let chunks;
  let recorder;
  let audioURL;
  let audioBlob;

  startButton.onclick = async function() {
    chunks = [];
    audioURL = undefined;
    audioBlob = undefined;

    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });

    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = (e) => chunks.push(e.data);
    recorder.onstop = async function() {
      audioBlob = new Blob(chunks, { type: 'audio/wav' });
      audioURL = URL.createObjectURL(audioBlob);

      // Send audio data to Python.
      const reader = new FileReader();
      reader.readAsDataURL(audioBlob);
      reader.onloadend = function() {
          const base64data = reader.result;

          // Send audio data to Python.
          google.colab.kernel.invokeFunction('notebook.audio_callback', [base64data], {});
      }
    };

    recorder.start();
    startButton.disabled = true;
    stopButton.disabled = false;
  }

  stopButton.onclick = function() {
    recorder.stop();

    startButton.disabled = false;
    stopButton.disabled = true;
  }
})();
"""

In [61]:
import base64
from google.colab import output
from IPython.display import Audio, Javascript

def audio_callback(base64_audio):
    header, base64_data = base64_audio.split(',')
    audio_data = base64.b64decode(base64_data)

    file_path = "recording.wav"
    with open(file_path, "wb") as f:
        f.write(audio_data)

    display(Audio(file_path))


# Register the callback with Colab
output.register_callback('notebook.audio_callback', audio_callback)

Javascript(MAKE_RECORD_BUTTONS_JS)

<IPython.core.display.Javascript object>

In [62]:
import librosa
import torch

audio_path = "recording.wav"
audio_data, sampling_rate = librosa.load(audio_path, sr=16000)

# Preprocess audio data
input_values = processor(audio_data, sampling_rate=16000, return_tensors="pt").input_values

# Perform model prediction
with torch.no_grad():
    logits = model(input_values).logits

# Decode prediction results
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

# Output results
print(f"Predicted Transcription: {transcription}")
display(Audio(audio_path))

Predicted Transcription: PLEASE LET ME KNOW THE ADDRESS OF THAT PERSON WHOSE NAME IS MOIL
